In [1]:
from Data_processing import import_images as i
from Classification import classification_class as classification
from Features_extraction import interest_point_detection as interest_point_detection
import pandas as pd

In [2]:
metrics_results_best_methods = {
        "accuracy" : {},
        "f1-score" : {},
        "recall" : {},
        "precision" : {},
        "roc_auc" : {}
        }

name_best_models = {}

# I. Point of interest detection

In [3]:
df_features = pd.DataFrame(interest_point_detection.method_SIFT(i.images))
df_Y = pd.DataFrame(i.Y)

data_SIFT = classification.DataProcessing(df_features, df_Y, stratified=False)
env_SIFT = classification.BinaryClassification(data_SIFT, average="macro")
metrics_results = env_SIFT.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_SIFT.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_SIFT.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["SIFT + BoW"] = best_method_name
metrics_results, predictions, models = env_SIFT.TrainTest()
env_SIFT.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_SIFT.get_metrics(models[best_method_name], "SIFT + BoW", metrics_results_best_methods)

=========1. Extract SIFT features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.80      0.78        86
           1       0.61      0.54      0.57        50

    accuracy                           0.71       136
   macro avg       0.68      0.67      0.67       136
weighted avg       0.70      0.71      0.70       136



In [4]:
df_features = pd.DataFrame(interest_point_detection.method_ORB(i.images))
df_Y = pd.DataFrame(i.Y)

data_ORB = classification.DataProcessing(df_features, df_Y, stratified=False)
env_ORB = classification.BinaryClassification(data_ORB, average="macro")
metrics_results = env_ORB.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_ORB.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_ORB.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["ORB + BoW"] = best_method_name
metrics_results, predictions, models = env_ORB.TrainTest()
env_ORB.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_ORB.get_metrics(models[best_method_name], "ORB + BoW", metrics_results_best_methods)

=========1. Extract ORB features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.48      0.55        86
           1       0.38      0.56      0.46        50

    accuracy                           0.51       136
   macro avg       0.52      0.52      0.50       136
weighted avg       0.55      0.51      0.52       136



# II. Edge detection

In [5]:
from Features_extraction import edge_detection as edges

df_features_edge = edges.features_df

data_edge = classification.DataProcessing(df_features_edge, df_Y, stratified=False)
env_edge = classification.BinaryClassification(data_edge, average="macro")
metrics_results = env_edge.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_edge.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_edge.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["EDGE"] = best_method_name
metrics_results, predictions, models = env_edge.TrainTest()
env_edge.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_edge.get_metrics(models[best_method_name], "EDGE", metrics_results_best_methods)


========= Extracting Edge Detection Features from Original Images =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.86      0.88        86
           1       0.77      0.82      0.80        50

    accuracy                           0.85       136
   macro avg       0.83      0.84      0.84       136
weighted avg       0.85      0.85      0.85       136



# III. Comparison after preprocessing

In [6]:
import numpy as np

original_images = [np.copy(img) for img in i.images]


## 1. First configuration

In [7]:
from Data_processing import data_preprocessing as dp
import cv2
import numpy as np

# Apply preprocessing to all images
print("=========  Applying Preprocessing to Images =========")

# Define target image size for resizing
target_size = (128, 128)

# Preprocessing parameters 
filter_method = "gaussian"
equalization_method = "hist"
apply_gamma = None
normalize_output = True
      
i.images = [np.copy(img) for img in original_images]

# Apply preprocessing to all images
preprocessed_images = [
    dp.preprocess_image(
        cv2.resize(img, target_size, interpolation=cv2.INTER_AREA), 
        method=filter_method, 
        equalization=equalization_method, 
        gamma=apply_gamma, 
        normalize=normalize_output
    ) 
    for img in i.images
]

# Convert list to NumPy array
preprocessed_images = np.array(preprocessed_images)

# Update dataset with preprocessed images
i.images = preprocessed_images

# Ensure images are in grayscale (avoid issues with SIFT)
i.images = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img for img in i.images])

# Ensure images are in uint8 format (SIFT requires this)
i.images = (i.images * 255).astype(np.uint8)

=========  Applying Preprocessing to Images =========


In [8]:
print("========= Extracting SIFT Features After Preprocessing =========")
df_features = pd.DataFrame(interest_point_detection.method_SIFT(i.images))
df_Y = pd.DataFrame(i.Y)

data_SIFT = classification.DataProcessing(df_features, df_Y, stratified=False)
env_SIFT = classification.BinaryClassification(data_SIFT, average="macro")
metrics_results = env_SIFT.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_SIFT.createMeansDataframe(metrics_results, labels)
best_method_name = env_SIFT.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for SIFT: {best_method_name}")
name_best_models["SIFT + BoW (Preprocessed - 1)"] = best_method_name
metrics_results, predictions, models = env_SIFT.TrainTest()
env_SIFT.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_SIFT.get_metrics(models[best_method_name], "SIFT + BoW (Preprocessed - 1)", metrics_results_best_methods)

print("========= Extracting ORB Features After Preprocessing =========")
df_features = pd.DataFrame(interest_point_detection.method_ORB(i.images))
df_Y = pd.DataFrame(i.Y)

data_ORB = classification.DataProcessing(df_features, df_Y, stratified=False)
env_ORB = classification.BinaryClassification(data_ORB, average="macro")
metrics_results = env_ORB.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_ORB.createMeansDataframe(metrics_results, labels)
best_method_name = env_ORB.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for ORB: {best_method_name}")
name_best_models["ORB + BoW (Preprocessed - 1)"] = best_method_name
metrics_results, predictions, models = env_ORB.TrainTest()
env_ORB.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_ORB.get_metrics(models[best_method_name], "ORB + BoW (Preprocessed - 1)", metrics_results_best_methods)


print("========= Extracting Edge Detection Features After Preprocessing =========")

df_features_edge = edges.extract_features_from_images(i.images)  # Recalculate features
df_Y = pd.DataFrame(i.Y)

data_edge = classification.DataProcessing(df_features_edge, df_Y, stratified=False)
env_edge = classification.BinaryClassification(data_edge, average="macro")
metrics_results = env_edge.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_edge.createMeansDataframe(metrics_results, labels)
best_method_name = env_edge.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for Edge Detection: {best_method_name}")
name_best_models["EDGE (Preprocessed - 1)"] = best_method_name
metrics_results, predictions, models = env_edge.TrainTest()
env_edge.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_edge.get_metrics(models[best_method_name], "EDGE (Preprocessed - 1)", metrics_results_best_methods)

========= Extracting SIFT Features After Preprocessing =========
=========1. Extract SIFT features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name for SIFT: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.76      0.79        86
           1       0.64      0.74      0.69        50

    accuracy                           0.75       136
   macro avg       0.74      0.75      0.74       136
weighted avg       0.76      0.75      0.75       136

========= Extracting ORB Features After Preprocessing =========
=========1. Extract ORB features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=

## 2. Second configuration

In [9]:
from Data_processing import data_preprocessing as dp
import cv2
import numpy as np

# Apply preprocessing to all images
print("=========  Applying Preprocessing to Images =========")

# Define target image size for resizing
target_size = (128, 128)

# Preprocessing parameters 
filter_method = "median"
equalization_method = "clahe"
apply_gamma = None
normalize_output = True

i.images = [np.copy(img) for img in original_images]

# Apply preprocessing to all images
preprocessed_images = [
    dp.preprocess_image(
        cv2.resize(img, target_size, interpolation=cv2.INTER_AREA), 
        method=filter_method, 
        equalization=equalization_method, 
        gamma=apply_gamma, 
        normalize=normalize_output
    ) 
    for img in i.images
]

# Convert list to NumPy array
preprocessed_images = np.array(preprocessed_images)

# Update dataset with preprocessed images
i.images = preprocessed_images

# Ensure images are in grayscale (avoid issues with SIFT)
i.images = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img for img in i.images])

# Ensure images are in uint8 format (SIFT requires this)
i.images = (i.images * 255).astype(np.uint8)

=========  Applying Preprocessing to Images =========


In [10]:
print("========= Extracting SIFT Features After Preprocessing =========")
df_features = pd.DataFrame(interest_point_detection.method_SIFT(i.images))
df_Y = pd.DataFrame(i.Y)

data_SIFT = classification.DataProcessing(df_features, df_Y, stratified=False)
env_SIFT = classification.BinaryClassification(data_SIFT, average="macro")
metrics_results = env_SIFT.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_SIFT.createMeansDataframe(metrics_results, labels)
best_method_name = env_SIFT.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for SIFT: {best_method_name}")
name_best_models["SIFT + BoW (Preprocessed - 2)"] = best_method_name
metrics_results, predictions, models = env_SIFT.TrainTest()
env_SIFT.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_SIFT.get_metrics(models[best_method_name], "SIFT + BoW (Preprocessed - 2)", metrics_results_best_methods)

print("========= Extracting ORB Features After Preprocessing =========")
df_features = pd.DataFrame(interest_point_detection.method_ORB(i.images))
df_Y = pd.DataFrame(i.Y)

data_ORB = classification.DataProcessing(df_features, df_Y, stratified=False)
env_ORB = classification.BinaryClassification(data_ORB, average="macro")
metrics_results = env_ORB.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_ORB.createMeansDataframe(metrics_results, labels)
best_method_name = env_ORB.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for ORB: {best_method_name}")
name_best_models["ORB + BoW (Preprocessed - 2)"] = best_method_name
metrics_results, predictions, models = env_ORB.TrainTest()
env_ORB.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_ORB.get_metrics(models[best_method_name], "ORB + BoW (Preprocessed - 2)", metrics_results_best_methods)


print("========= Extracting Edge Detection Features After Preprocessing =========")

df_features_edge = edges.extract_features_from_images(i.images)  # Recalculate features
df_Y = pd.DataFrame(i.Y)

data_edge = classification.DataProcessing(df_features_edge, df_Y, stratified=False)
env_edge = classification.BinaryClassification(data_edge, average="macro")
metrics_results = env_edge.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_edge.createMeansDataframe(metrics_results, labels)
best_method_name = env_edge.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for Edge Detection: {best_method_name}")
name_best_models["EDGE (Preprocessed - 2)"] = best_method_name
metrics_results, predictions, models = env_edge.TrainTest()
env_edge.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_edge.get_metrics(models[best_method_name], "EDGE (Preprocessed - 2)", metrics_results_best_methods)

========= Extracting SIFT Features After Preprocessing =========
=========1. Extract SIFT features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name for SIFT: RandomForest
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.69      0.76        86
           1       0.60      0.80      0.68        50

    accuracy                           0.73       136
   macro avg       0.73      0.74      0.72       136
weighted avg       0.76      0.73      0.73       136

========= Extracting ORB Features After Preprocessing =========
=========1. Extract ORB features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =====

## 3. Third configuration

In [18]:
from Data_processing import data_preprocessing as dp
import cv2
import numpy as np

# Apply preprocessing to all images
print("=========  Applying Preprocessing to Images =========")

# Define target image size for resizing
target_size = (128, 128)

# Preprocessing parameters 
filter_method = "bilateral"
equalization_method = "clahe"
apply_gamma = 1.5  # Adjust gamma to brighten images
normalize_output = True

i.images = [np.copy(img) for img in original_images]

# Apply preprocessing to all images
preprocessed_images = [
    dp.preprocess_image(
        cv2.resize(img, target_size, interpolation=cv2.INTER_AREA), 
        method=filter_method, 
        equalization=equalization_method, 
        gamma=apply_gamma, 
        normalize=normalize_output
    ) 
    for img in i.images
]

# Convert list to NumPy array
preprocessed_images = np.array(preprocessed_images)

# Update dataset with preprocessed images
i.images = preprocessed_images

# Ensure images are in grayscale (avoid issues with SIFT)
i.images = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img for img in i.images])

# Ensure images are in uint8 format (SIFT requires this)
i.images = (i.images * 255).astype(np.uint8)

=========  Applying Preprocessing to Images =========


In [19]:
print("========= Extracting SIFT Features After Preprocessing =========")
df_features = pd.DataFrame(interest_point_detection.method_SIFT(i.images))
df_Y = pd.DataFrame(i.Y)

data_SIFT = classification.DataProcessing(df_features, df_Y, stratified=False)
env_SIFT = classification.BinaryClassification(data_SIFT, average="macro")
metrics_results = env_SIFT.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_SIFT.createMeansDataframe(metrics_results, labels)
best_method_name = env_SIFT.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for SIFT: {best_method_name}")
name_best_models["SIFT + BoW (Preprocessed - 3)"] = best_method_name
metrics_results, predictions, models = env_SIFT.TrainTest()
env_SIFT.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_SIFT.get_metrics(models[best_method_name], "SIFT + BoW (Preprocessed - 3)", metrics_results_best_methods)

print("========= Extracting ORB Features After Preprocessing =========")
df_features = pd.DataFrame(interest_point_detection.method_ORB(i.images))
df_Y = pd.DataFrame(i.Y)


# Debugging
print(f"Shape de df_features (features extraites) : {df_features.shape}")
print(f"Shape de df_Y (labels) : {df_Y.shape}")
valid_indices = [idx for idx, feature in enumerate(interest_point_detection.method_ORB(i.images)) if feature is not None and len(feature) > 0]
df_features = pd.DataFrame([feature for idx, feature in enumerate(interest_point_detection.method_ORB(i.images)) if idx in valid_indices])
df_Y = df_Y.iloc[valid_indices].reset_index(drop=True)

data_ORB = classification.DataProcessing(df_features, df_Y, stratified=False)
env_ORB = classification.BinaryClassification(data_ORB, average="macro")
metrics_results = env_ORB.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_ORB.createMeansDataframe(metrics_results, labels)
best_method_name = env_ORB.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for ORB: {best_method_name}")
name_best_models["ORB + BoW (Preprocessed - 3)"] = best_method_name
metrics_results, predictions, models = env_ORB.TrainTest()
env_ORB.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_ORB.get_metrics(models[best_method_name], "ORB + BoW (Preprocessed - 3)", metrics_results_best_methods)

print("========= Extracting Edge Detection Features After Preprocessing =========")

df_features_edge = edges.extract_features_from_images(i.images)  # Recalculate features
df_Y = pd.DataFrame(i.Y)

data_edge = classification.DataProcessing(df_features_edge, df_Y, stratified=False)
env_edge = classification.BinaryClassification(data_edge, average="macro")
metrics_results = env_edge.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_edge.createMeansDataframe(metrics_results, labels)
best_method_name = env_edge.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for Edge Detection: {best_method_name}")
name_best_models["EDGE (Preprocessed - 3)"] = best_method_name
metrics_results, predictions, models = env_edge.TrainTest()
env_edge.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_edge.get_metrics(models[best_method_name], "EDGE (Preprocessed - 3)", metrics_results_best_methods)

========= Extracting SIFT Features After Preprocessing =========
=========1. Extract SIFT features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name for SIFT: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.80      0.83        86
           1       0.69      0.76      0.72        50

    accuracy                           0.79       136
   macro avg       0.77      0.78      0.78       136
weighted avg       0.79      0.79      0.79       136

========= Extracting ORB Features After Preprocessing =========
=========1. Extract ORB features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=

## 4. Fourth configuration

In [20]:
from Data_processing import data_preprocessing as dp
import cv2
import numpy as np

# Apply preprocessing to all images
print("=========  Applying Preprocessing to Images =========")

# Define target image size for resizing
target_size = (128, 128)

# Preprocessing parameters 
filter_method = "bilateral"
equalization_method = "hist"
apply_gamma = 1.2
normalize_output = True

i.images = [np.copy(img) for img in original_images]

# Apply preprocessing to all images
preprocessed_images = [
    dp.preprocess_image(
        cv2.resize(img, target_size, interpolation=cv2.INTER_AREA), 
        method=filter_method, 
        equalization=equalization_method, 
        gamma=apply_gamma, 
        normalize=normalize_output
    ) 
    for img in i.images
]

# Convert list to NumPy array
preprocessed_images = np.array(preprocessed_images)

# Update dataset with preprocessed images
i.images = preprocessed_images

# Ensure images are in grayscale (avoid issues with SIFT)
i.images = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img for img in i.images])

# Ensure images are in uint8 format (SIFT requires this)
i.images = (i.images * 255).astype(np.uint8)

=========  Applying Preprocessing to Images =========


In [21]:
print("========= Extracting SIFT Features After Preprocessing =========")
df_features = pd.DataFrame(interest_point_detection.method_SIFT(i.images))
df_Y = pd.DataFrame(i.Y)

data_SIFT = classification.DataProcessing(df_features, df_Y, stratified=False)
env_SIFT = classification.BinaryClassification(data_SIFT, average="macro")
metrics_results = env_SIFT.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_SIFT.createMeansDataframe(metrics_results, labels)
best_method_name = env_SIFT.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for SIFT: {best_method_name}")
name_best_models["SIFT + BoW (Preprocessed - 4)"] = best_method_name
metrics_results, predictions, models = env_SIFT.TrainTest()
env_SIFT.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_SIFT.get_metrics(models[best_method_name], "SIFT + BoW (Preprocessed - 4)", metrics_results_best_methods)

print("========= Extracting ORB Features After Preprocessing =========")
df_features = pd.DataFrame(interest_point_detection.method_ORB(i.images))
df_Y = pd.DataFrame(i.Y)

# Debugging
print(f"Shape de df_features (features extraites) : {df_features.shape}")
print(f"Shape de df_Y (labels) : {df_Y.shape}")
valid_indices = [idx for idx, feature in enumerate(interest_point_detection.method_ORB(i.images)) if feature is not None and len(feature) > 0]
df_features = pd.DataFrame([feature for idx, feature in enumerate(interest_point_detection.method_ORB(i.images)) if idx in valid_indices])
df_Y = df_Y.iloc[valid_indices].reset_index(drop=True)


data_ORB = classification.DataProcessing(df_features, df_Y, stratified=False)
env_ORB = classification.BinaryClassification(data_ORB, average="macro")
metrics_results = env_ORB.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_ORB.createMeansDataframe(metrics_results, labels)
best_method_name = env_ORB.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for ORB: {best_method_name}")
name_best_models["ORB + BoW (Preprocessed - 4)"] = best_method_name
metrics_results, predictions, models = env_ORB.TrainTest()
env_ORB.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_ORB.get_metrics(models[best_method_name], "ORB + BoW (Preprocessed - 4)", metrics_results_best_methods)


print("========= Extracting Edge Detection Features After Preprocessing =========")

df_features_edge = edges.extract_features_from_images(i.images)  # Recalculate features
df_Y = pd.DataFrame(i.Y)

data_edge = classification.DataProcessing(df_features_edge, df_Y, stratified=False)
env_edge = classification.BinaryClassification(data_edge, average="macro")
metrics_results = env_edge.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_edge.createMeansDataframe(metrics_results, labels)
best_method_name = env_edge.get_best_method(results_test_KFold, "F1-score", ens="Test")

print(f"Best method name for Edge Detection: {best_method_name}")
name_best_models["EDGE (Preprocessed - 4)"] = best_method_name
metrics_results, predictions, models = env_edge.TrainTest()
env_edge.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_edge.get_metrics(models[best_method_name], "EDGE (Preprocessed - 4)", metrics_results_best_methods)

========= Extracting SIFT Features After Preprocessing =========
=========1. Extract SIFT features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name for SIFT: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.71      0.73        86
           1       0.55      0.62      0.58        50

    accuracy                           0.68       136
   macro avg       0.66      0.66      0.66       136
weighted avg       0.69      0.68      0.68       136

========= Extracting ORB Features After Preprocessing =========
=========1. Extract ORB features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=

# IV. Overall results

In [22]:
df_best_models_all_datasets_macro_f1 = pd.DataFrame(metrics_results_best_methods)
df_best_models_all_datasets_macro_f1.insert(0, 'Best models', name_best_models)
df_best_models_all_datasets_macro_f1.style.highlight_max(axis=0, subset=df_best_models_all_datasets_macro_f1.columns[1:])

,Best models,accuracy,f1-score,recall,precision,roc_auc
SIFT + BoW,LogReg,0.705882,0.674874,0.671163,0.681818,0.671163
ORB + BoW,LogReg,0.507353,0.502810,0.518372,0.517178,0.518372
EDGE,LogReg,0.845588,0.835928,0.840233,0.832576,0.840233
SIFT + BoW (Preprocessed - 1),LogReg,0.750000,0.738934,0.747907,0.735632,0.747907
ORB + BoW (Preprocessed - 1),LogReg,0.669118,0.645649,0.646279,0.645098,0.646279
EDGE (Preprocessed - 1),LogReg,0.808824,0.804598,0.827907,0.805195,0.827907
SIFT + BoW (Preprocessed - 2),RandomForest,0.727941,0.722526,0.743023,0.726044,0.743023
ORB + BoW (Preprocessed - 2),RandomForest,0.595588,0.578995,0.583953,0.579463,0.583953
EDGE (Preprocessed - 2),LogReg,0.823529,0.814503,0.822791,0.809821,0.822791
SIFT + BoW (Preprocessed - 3),LogReg,0.786765,0.775078,0.781163,0.771380,0.781163
